In [ ]:
!pip install requests beautifulsoup4 langchain-openai

In [11]:
# Instalando bibliotecas necessárias
!pip install requests beautifulsoup4 langchain-openai

# Importando bibliotecas
import requests
from bs4 import BeautifulSoup
from langchain_openai.chat_models.azure import AzureChatOpenAI

# Configurações do Azure OpenAI
AZURE_OPENAI_API_KEY = 'sua_chave_api'
AZURE_OPENAI_ENDPOINT = 'https://seu-recurso.openai.azure.com/'
AZURE_OPENAI_DEPLOYMENT_NAME = 'nome_do_seu_modelo'
AZURE_OPENAI_API_VERSION = '2024-08-01-preview'

# Inicializando o cliente Azure OpenAI com o modelo de chat e a versão 2024-08-01-preview
llm = AzureChatOpenAI(
    azure_endpoint= AZURE_OPENAI_ENDPOINT,
    api_key= AZURE_OPENAI_API_KEY,
    api_version= AZURE_OPENAI_API_VERSION,
    azure_deployment= AZURE_OPENAI_DEPLOYMENT_NAME,
    max_retries=0
)

def get_clean_text_from_url(url):
    """Extrai e limpa o texto do artigo a partir da URL fornecida."""
    response = requests.get(url)
    if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')
      for script_or_style in soup(['script', 'style']):
        script_or_style.decompose()
      texto = soup.get_text(separator= ' ')
      #Limpar texto
      linhas = (line.strip() for line in texto.splitlines())
      parts = (phrase.strip() for line in linhas for phrase in line.split(' '))
      texto_limpo = '\n'.join(part for part in parts if part)
      return texto_limpo
    else:
      print(f'Failed to fetch the URL. Status code: {response.status_code}')
      return None
    text = soup.get_text()
    return text

def translate_text(text, target_language):
    """Traduz o texto para o idioma solicitado pelo usuário usando Azure OpenAI."""
    prompt = f"Translate the following text to {target_language}:\n\n{text}"
    response = llm(prompt)
    return response

def format_text_as_markdown(text):
    """Formata o texto em Markdown usando o Azure OpenAI para estruturação."""
    prompt = f"Convert the following text into a well-structured Markdown format:\n\n{text}"
    response = llm(prompt)
    return response

# URL do artigo e idioma de destino
url = input("Digite a URL do artigo: ")  # Permite que o usuário insira a URL do artigo
target_language = input("Digite o idioma para tradução (ex: 'Portuguese', 'English', 'Spanish', etc.): ")  # Permite que o usuário escolha o idioma

# 1. Obter o texto do artigo limpo
article_text = get_clean_text_from_url(url)

# 2. Traduzir o texto
translated_text = translate_text(article_text, target_language)

# 3. Formatar o texto em Markdown
markdown_text = format_text_as_markdown(translated_text)

# Exibindo o resultado
print("Texto em Markdown:\n")
print(markdown_text.content)

Digite a URL do artigo: https://dev.to/kenakamu/azure-open-ai-in-vnet-3alo
Digite o idioma para tradução (ex: 'Portuguese', 'English', 'Spanish', etc.): pt-br
Texto em Markdown:

```markdown
# Azure Open AI na VNet

**Postado por:** Kenichiro Nakamura  
**Data:** 12 de outubro, 2023

## Navegação

- [Entrar](#)
- [Criar conta](#)
- [Comunidade DEV](#)
- [Buscar](#)

---

## Introdução

Os modelos GPT são hospedados em vários fornecedores de serviço no momento, e Microsoft Azure é um deles. Embora os modelos em si sejam os mesmos, existem muitas diferenças, incluindo: custo, funcionalidades, tipo de modelos e versões, localização geográfica, segurança, suporte, etc. 

Um dos aspectos mais importantes quando usamos isso em um Ambiente Empresarial é, claro, a segurança. Ao usar os recursos de segurança de rede do Azure com Azure Open AI, os clientes podem consumir o serviço do Open AI de e dentro do VNet, portanto, nenhuma informação flui publicamente.

## Implantação

### Exemplo de Impl